In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import requests, json

* read DM dataset

In [3]:
data_DM = pd.read_json('/content/drive/MyDrive/Accountstory/data/people_list_DM.json')

In [6]:
data_DM

,name,position,email,linkedin_url,level,relevant_title,start_date,end_date,company_linkedin_urls,company_website,company,phone_number,location,full_address,source,source_companies,id,is_core_department,in_valid_email_domain,valid_email_domain,is_relevant_title_present,department,account_map_category,stakeholder_category
0,jesse barker,co-founder and operations,jesse@zennify.com,https://www.linkedin.com/in/jessegbarker,[owner],None,2014-03,None,"[linkedin.com/company/zennify, linkedin.com/co...",zennify.com,zennify,1.208298e+10,Nampa,"Nampa, Idaho, United States",pdl_bulk_api,{'572237': ['Payroll Software']},531411.0,True,False,False,False,Business Operations,verified_executive_level_buyer,influencer
1,tami miller,chief operating officer,tami@zennify.com,https://www.linkedin.com/in/tamimiller,[cxo],"director, delivery and solution design",2020-01,None,"[linkedin.com/company/zennify, linkedin.com/co...",zennify.com,zennify,1.937559e+10,Meridian,"Meridian, Idaho, United States",pdl_bulk_api,{'525162': ['Cloud Telephony Tools']},653917.0,True,False,False,False,Business Operations,verified_executive_level_buyer,decision_maker
2,ashley richter,head of marketing programs and operations,arichter@authentic8.com,https://www.linkedin.com/in/ashley-richter-a94...,[director],None,2018-03,None,"[linkedin.com/company/authentic8, linkedin.com...",authentic8.com,authentic8,NaN,Indianapolis,"Indianapolis, Indiana, United States",pdl_bulk_api,{'402368': ['Video Software Tools']},774355.0,True,False,False,False,Marketing,director_level_buyer,decision_maker
3,drew paik,"european business development for authentic8, ...",drew@authentic8.com,https://www.linkedin.com/in/drewpaik,[],marketing director,2019-01,None,"[linkedin.com/company/authentic8, linkedin.com...",authentic8.com,authentic8,1.650352e+10,Washington,"Washington, District Of Columbia, United States",pdl_bulk_api,{'243484': ['Payment & Billing Software Tools']},942496.0,True,False,False,False,Marketing,verified_individual_contributor_influencer,influencer
4,yee sim,finance controller,ysim@authentic8.com,https://www.linkedin.com/in/yee-sim-9091704b,[],None,2020-04,None,"[linkedin.com/company/authentic8, linkedin.com...",authentic8.com,authentic8,NaN,San Mateo,"San Mateo, California, United States",pdl_bulk_api,{'795365': ['Payment & Billing Software Tools']},532214.0,True,False,False,False,Finance & Accounting,verified_individual_contributor_influencer,influencer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15652,chris davis,senior strategic account executive,chris.davis@creatoriq.com,https://www.linkedin.com/in/chris-davis-4b760170,[senior],None,2021-03,None,"[linkedin.com/company/creatoriq, linkedin.com/...",creatoriq.com,creatoriq,1.646927e+10,Los Angeles,"Los Angeles, California, United States",pdl_bulk_api,"{'953730': ['Sales Enablement Tools'], '922156...",285343.0,True,False,False,False,Sales,verified_individual_contributor_influencer,influencer
15653,julia kearney,senior director of strategic accounts,julia@creatoriq.com,https://www.linkedin.com/search/results/all/?k...,"[senior, director]",None,None,None,"[linkedin.com/company/creatoriq, linkedin.com/...",creatoriq.com,creatoriq,NaN,None,None,pdl_bulk_api,{'953730': ['Sales Enablement Tools']},275247.0,True,False,False,False,Sales,director_level_buyer,decision_maker
15654,bhavin desai,vice president product strategy,bhavin@creatoriq.com,https://www.linkedin.com/search/results/all/?k...,[vp],None,None,None,"[linkedin.com/company/creatoriq, linkedin.com/...",creatoriq.com,creatoriq,NaN,None,None,pdl_bulk_api,{'922156': ['Partner Management Software']},353509.0,True,False,False,False,Product,verified_executive_level_buyer,decision_maker
15655,natasha shifrin,"director, sales development and strategy",natasha@creatoriq.com,https://www.linkedin.com/search/results/all/?k...,[director],None,None,None,"[linkedin.com/company/creatoriq, linkedin.com/...",creatoriq.com,creatoriq,NaN,None

In [5]:
def call_enrich_api(company, name):

    API_KEY = '1229cba1bb8dc3f5a3a3e93ec0e2826ec84280767f3109502966418dbfd7f896'

    pdl_url = "https://api.peopledatalabs.com/v5/person/enrich"

    params = {
        "api_key": API_KEY,
        "company": [company],   
        "first_name": [name.split()[0]],
        "last_name": [name.split()[1]]
    }

    json_response = requests.get(pdl_url,  params=params).json()

    if json_response["status"] == 200:
        return json_response['data']

        #print(
        #    record['work_email'],
        #    record['full_name'],
        #    record['job_title'],
        #    record['job_company_name']
        #  )

        #print(f"successfully enriched profile with pdl data")

        # Save enrichment data to json file
        #with open("my_pdl_enrichment.jsonl", "w") as out:
        #    out.write(json.dumps(record) + "\n")
    else:
        print("Enrichment unsuccessful. See error and try again.")
        print("error:", json_response)

In [17]:
N_DATAPOINTS = 400
start = 1000

df = pd.DataFrame()
i = 0
for company, name in zip(data_DM[start:start+N_DATAPOINTS]['company'],data_DM[start:start+N_DATAPOINTS]['name']):
    try:
        records = call_enrich_api(company, name)
        df_temp = pd.DataFrame.from_dict(records, orient='index').transpose()
        if not df.empty:
            df = df.append(df_temp, ignore_index = True)
        else:
            df = df_temp
        if i%20 == 0:
            print(i)
        i += 1
    except:
        pass

0
Enrichment unsuccessful. See error and try again.
error: {'status': 404, 'error': {'type': 'not_found', 'message': 'No records were found matching your request'}}
20
Enrichment unsuccessful. See error and try again.
error: {'status': 404, 'error': {'type': 'not_found', 'message': 'No records were found matching your request'}}
40
60
80
100
120
Enrichment unsuccessful. See error and try again.
error: {'status': 404, 'error': {'type': 'not_found', 'message': 'No records were found matching your request'}}
Enrichment unsuccessful. See error and try again.
error: {'status': 404, 'error': {'type': 'not_found', 'message': 'No records were found matching your request'}}
140
160
Enrichment unsuccessful. See error and try again.
error: {'status': 404, 'error': {'type': 'not_found', 'message': 'No records were found matching your request'}}
180
200
Enrichment unsuccessful. See error and try again.
error: {'status': 404, 'error': {'type': 'not_found', 'message': 'No records were found matching 

In [18]:
df.drop('id', axis=1, inplace=True)

In [19]:
df.shape

(389, 71)

In [20]:
df_old = pd.read_json('/content/drive/MyDrive/Accountstory/data/people_list_DM_enriched_20210930.json')

In [21]:
df = df.append(df_old)
df.shape

(1366, 71)

In [22]:
df.reset_index(inplace=True)
df.drop('index',axis=1, inplace=True)
df

,full_name,first_name,middle_initial,middle_name,last_name,gender,birth_year,birth_date,linkedin_url,linkedin_username,linkedin_id,facebook_url,facebook_username,facebook_id,twitter_url,twitter_username,github_url,github_username,work_email,personal_emails,mobile_phone,industry,job_title,job_title_role,job_title_sub_role,job_title_levels,job_company_id,job_company_name,job_company_website,job_company_size,job_company_founded,job_company_industry,job_company_linkedin_url,job_company_linkedin_id,job_company_facebook_url,job_company_twitter_url,job_company_location_name,job_company_location_locality,job_company_location_metro,job_company_location_region,job_company_location_geo,job_company_location_street_address,job_company_location_address_line_2,job_company_location_postal_code,job_company_location_country,job_company_location_continent,job_last_updated,job_start_date,location_name,location_locality,location_metro,location_region,location_country,location_continent,location_street_address,location_address_line_2,location_postal_code,location_geo,location_last_updated,phone_numbers,emails,interests,skills,location_names,regions,countries,street_addresses,experience,education,profiles,version_status
0,carlos capelari,carlos,a,augusto,capelari,male,None,None,linkedin.com/in/carlos-augusto-capelari,carlos-augusto-capelari,420732659,facebook.com/carlosaugusto.capelari,carlosaugusto.capelari,100001711652804,None,None,None,None,None,[carlosaugustocapelari@gmail.com],None,computer software,data analyst,engineering,data,[],pipefy,pipefy,pipefy.com,201-500,2014,computer software,linkedin.com/company/pipefy,5202118,facebook.com/pipefy,twitter.com/pipefy,"california, united states",None,None,california,None,None,None,None,united states,north america,2021-06-01,2019-09,"curitiba, parana, brazil",curitiba,None,parana,brazil,south america,None,None,None,None,2021-06-01,[+14796393824],[{'address': 'carlosaugustocapelari@gmail.com'...,[economic empowerment],"[microsoft excel, data analysis, sql, indicado...","[joinville, santa catarina, brazil, curitiba, ...","[parana, brazil, santa catarina, brazil]",[brazil],[],"[{'company': {'name': 'maxmilhas', 'size': '20...",[{'school': {'name': 'udesc - universidade do ...,"[{'network': 'linkedin', 'id': '420732659', 'u...","{'status': 'updated', 'contains': [], 'previou..."
1,felipe cibin,felipe,None,None,cibin,male,None,None,linkedin.com/in/felipecibin,felipecibin,366326968,facebook.com/fecibin,fecibin,100002645444534,None,None,None,None,felipe@pipefy.com,[],None,computer software,account growth manager,marketing,None,[manager],pipefy,pipefy,pipefy.com,201-500,2014,computer software,linkedin.com/company/pipefy,5202118,facebook.com/pipefy,twitter.com/pipefy,"california, united states",None,None,california,None,None,None,None,united states,north america,2021-06-01,2020-01,"curitiba, parana, brazil",curitiba,None,parana,brazil,south america,None,None,None,None,2021-06-01,[+17864565067],"[{'address': 'felipe@pipefy.com', 'type': 'cur...","[politics, science and technology, education, ...","[microsoft excel, customer service, management...","[san francisco, california, united states, cur...","[california, united states, parana, brazil]","[united states, brazil]",[],"[{'company': {'name': 'pipefy', 'size': '201-5...","[{'school': {'name': 'stanford university', 't...","[{'network': 'linkedin', 'id': '366326968', 'u...","{'status': 'updated', 'contains': [], 'previou..."
2,milton costa,milton,None,None,costa,male,None,None,linkedin.com/in/milton-costa,milton-costa,190734112,None,None,None,None,None,None,None,milton.costa@pipefy.com,[],None,computer software,sre manager,None,None,[manager],pipefy,pipefy,pipefy.com,201-500,2014,computer software,linkedin.com/company/pipefy,5202118,facebook.com/pipefy,twitter.com/pipefy,"california, united states",None,None,california,None,None,None,None,united states,north america,2021-06-01,2020-08,"curitiba, parana, brazil",curitiba,None,parana,brazil,south america,None,None

In [29]:
df.drop_duplicates(subset=['full_name','linkedin_id','industry','job_title'], inplace=True)

In [30]:
df.to_json('/content/drive/MyDrive/Accountstory/data/people_list_DM_enriched_20210930.json')